In [7]:
import pandas as pd
pd.set_option('use_inf_as_na', True)
import numpy as np
import re

**Develop bot detection features**

In [2]:
# load in the 5000 random samples
accounts_df = pd.read_csv("5000_accounts_climate.csv")
accounts_df.head()

,id,name,username,location,url,description,verified,followers,friends,favourites_count,statuses_count,created_at,default_profile,default_profile_image
0,1098803589609189376,💧The Cranky Croation,JohnSarich2,NaN,NaN,My First ever vote was for Gough Whitlam. Left...,False,430,291,14866,6039,2019-02-22 04:36:05,True,False
1,191393940,FranS #RejoinEU #ElectoralReform 🇬🇧🇪🇺🇩🇪,FranS199,Earth,NaN,"Love good food, good wine, great company. Hate...",False,4643,4904,257039,41696,2010-09-16 10:30:21,True,False
2,1185167241819676673,mbnvcxz,mbnvcxz2,NaN,NaN,NaN,False,0,3,1,8,2019-10-18 12:14:42,True,True
3,1220868357647368193,Noles-4-Life-In-SC,Noles4LifeInSC,NaN,NaN,Florida native living in Upstate South Carolin...,False,628,630,1852,1862,2020-01-25 00:39:20,True,False
4,347142932,Life Cycle UK,LifeCycleUKteam,Bristol,http://t.co/oCrHfe7xAg,Life Cycle UK is a Bristol-based charity that ...,False,3105,1645,3039,5349,2011-08-02 10:21:33,False,False


In [3]:
accounts_df['default_profile'] = accounts_df['default_profile']*1
accounts_df['default_profile_image'] = accounts_df['default_profile_image']*1

In [4]:
# days since account has been open
accounts_df['date_created'], accounts_df['time'] = accounts_df['created_at'].str.split(' ', 1).str
accounts_df['date_created'] = pd.to_datetime(accounts_df.date_created, format='%Y-%m-%d')
accounts_df['day'] = ('2020-03-12')
accounts_df['day'] = pd.to_datetime(accounts_df.day, format='%Y-%m-%d')
accounts_df['days_active'] = (accounts_df['day'] - accounts_df['date_created']).dt.days

if 'day' in accounts_df:
    accounts_df = accounts_df.drop(columns=['time','day','created_at'])

else:
    accounts_df.head()

/Users/jackpawson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  


In [5]:
# social figures in relation to account age
accounts_df['followers_age'] = accounts_df['followers'] / accounts_df['days_active']
accounts_df['following_age'] = accounts_df['friends'] / accounts_df['days_active']
accounts_df['favourites_age'] = accounts_df['favourites_count'] / accounts_df['days_active']
accounts_df['tweets_age'] = accounts_df['statuses_count'] / accounts_df['days_active']

# following to follower ratio
accounts_df['followers_ratio'] = accounts_df['friends'] / accounts_df['followers']

accounts_df.head(3)

,id,name,username,location,url,description,verified,followers,friends,favourites_count,statuses_count,default_profile,default_profile_image,date_created,days_active,followers_age,following_age,favourites_age,tweets_age,followers_ratio
0,1098803589609189376,💧The Cranky Croation,JohnSarich2,NaN,NaN,My First ever vote was for Gough Whitlam. Left...,False,430,291,14866,6039,1,0,2019-02-22,384,1.119792,0.757812,38.713542,15.726562,0.676744
1,191393940,FranS #RejoinEU #ElectoralReform 🇬🇧🇪🇺🇩🇪,FranS199,Earth,NaN,"Love good food, good wine, great company. Hate...",False,4643,4904,257039,41696,1,0,2010-09-16,3465,1.339971,1.415296,74.181530,12.033478,1.056214
2,1185167241819676673,mbnvcxz,mbnvcxz2,NaN,NaN,NaN,False,0,3,1,8,1,1,2019-10-18,146,0.000000,0.020548,0.006849,0.054795,NaN


In [8]:
# username features - can only use A-Z, 0-9 and _ + not case-sensitive
accounts_df['username_char_len'] = accounts_df['username'].str.len()

# length of username in comparison to length of real name
accounts_df['name_ratio'] = accounts_df['name'].str.len() / accounts_df['username'].str.len()
    # amount of numbers in username
username_int = []    
for i in accounts_df['username']:
    numbers = sum(c.isdigit() for c in i)
    username_int.append(numbers)
accounts_df['username_int'] = username_int

    # amount of characters
username_char = []    
for i in accounts_df['username']:
    char = sum(c.isalpha() for c in i)
    username_char.append(char)
accounts_df['username_char'] = username_char

    # underscores 
accounts_df['username_other'] = accounts_df['username_char_len'] - (accounts_df['username_int'] + 
                                                                       accounts_df['username_char'])

# amount of numbers at the end of username if there is any
results = []

for i in accounts_df['username']:
    m = re.search(r'\d+$', i)
    if m is None:
        results.append(0)
    elif m is not None:
        count = m.group()
        results.append(len(count))

accounts_df['username_int_end'] = results

# amount of numbers in display name (not common for real users, but maybe for businesses!)
name_int = []    
for i in accounts_df['name']:
    numbers = sum(c.isdigit() for c in i)
    name_int.append(numbers)
accounts_df['name_int'] = name_int

# is there any cases of bots using techniques to make account details appear real, emoji flags, hashtags etc?
accounts_df.head()

,id,name,username,location,url,description,verified,followers,friends,favourites_count,...,favourites_age,tweets_age,followers_ratio,username_char_len,name_ratio,username_int,username_char,username_other,username_int_end,name_int
0,1098803589609189376,💧The Cranky Croation,JohnSarich2,NaN,NaN,My First ever vote was for Gough Whitlam. Left...,False,430,291,14866,...,38.713542,15.726562,0.676744,11,1.818182,1,10,0,1,0
1,191393940,FranS #RejoinEU #ElectoralReform 🇬🇧🇪🇺🇩🇪,FranS199,Earth,NaN,"Love good food, good wine, great company. Hate...",False,4643,4904,257039,...,74.181530,12.033478,1.056214,8,4.875000,3,5,0,3,0
2,1185167241819676673,mbnvcxz,mbnvcxz2,NaN,NaN,NaN,False,0,3,1,...,0.006849,0.054795,NaN,8,0.875000,1,7,0,1,0
3,1220868357647368193,Noles-4-Life-In-SC,Noles4LifeInSC,NaN,NaN,Florida native living in Upstate South Carolin...,False,628,630,1852,...,39.404255,39.617021,1.003185,14,1.285714,1,13,0,0,1
4,347142932,Life Cycle UK,LifeCycleUKteam,Bristol,http://t.co/oCrHfe7xAg,Life Cycle UK is a Bristol-based charity that ...,False,3105,1645,3039,...,0.966296,1.700795,0.529791,15,0.866667,0,15,0,0,0


In [9]:
# need to find a way of improving this method ...
def remove_emojis(string):
    return string.encode('ascii', 'ignore').decode('ascii')

In [10]:
# analysis of description...

In [28]:
# analysis of username and screenname similarity ...

**Looking at the tweets to create some tweet frequency features**

In [12]:
# tweets
df = pd.read_csv("tweets_5000_accounts.csv").set_index('screen_name')
df['date'] = pd.to_datetime(df.date, format='%d/%m/%Y')

In [13]:
df.head()

,date,time,text,tweetid,datetime
screen_name,,,,,
JohnSarich2,2020-03-25,05:32:00,RT @MadamEarth: FFS. How can we just sit at ho...,1.240000e+14,25/03/2020 05:32:00
JohnSarich2,2020-03-25,05:31:00,RT @daveyk317: If it’s true that the Ruby Prin...,1.240000e+14,25/03/2020 05:31:00
JohnSarich2,2020-03-25,05:31:00,I suppose this will be Morrison's thought bubb...,1.240000e+14,25/03/2020 05:31:00
JohnSarich2,2020-03-25,05:28:00,@Pen2Paper43 @icred1 Yep. They're not up to it.,1.240000e+14,25/03/2020 05:28:00
JohnSarich2,2020-03-25,05:27:00,RT @GayeCrispin: #auspol #COVID19Aus #covid19a...,1.240000e+14,25/03/2020 05:27:00


In [14]:
screen_names = df.index.unique().tolist()
print(len(screen_names))

4469


In [15]:
# how many of recent 200 tweets were not unique.
non_unique_tweets = []

for i in screen_names:
    user = df.loc[df.index == i]
    groupby_df = user.groupby('text').count()
    groupby_grt = groupby_df.loc[groupby_df.tweetid > 1]
    greater_than_1 = sum(groupby_grt['tweetid'])
    non_unique_tweets.append(greater_than_1)
    
    
print(len(non_unique_tweets))
results = pd.DataFrame({'screen_name':screen_names,'non_unique_tweets':non_unique_tweets})

4469


In [16]:
# average number of tweets per day - from last 200
average_tweets_day = []

for i in screen_names:
    user = df.loc[df.index == i]
    dates_group = user.groupby('date').count()
    mean_tweets = np.round(dates_group['tweetid'].mean(),decimals=0)
    average_tweets_day.append(mean_tweets)
    
print(len(average_tweets_day))
average_tweets_results = pd.DataFrame({'screen_name':screen_names,'average_tweets_day':average_tweets_day})

4469


In [17]:
results = results.merge(average_tweets_results,left_on='screen_name',right_on='screen_name')

In [18]:
results.head()

,screen_name,non_unique_tweets,average_tweets_day
0,JohnSarich2,0,40.0
1,FranS199,0,33.0
2,mbnvcxz2,0,1.0
3,Noles4LifeInSC,0,33.0
4,LifeCycleUKteam,0,3.0


In [19]:
# number of tweets per day - relative to account age --ADD TO FULL USER TWEETS

In [20]:
# average time between tweets
minutes_between_tweets_results = []

for i in screen_names:
    
    # slice the df by username
    test = pd.DataFrame(df.loc[df.index == i])
    test = test.set_index('datetime')
    
    # create colmn for tweet time and consecutive tweet time
    test['tvalue'] = test.index
    test['tvalue'] = pd.to_datetime(test.tvalue,format = '%d/%m/%Y %H:%M:%S')
    test['shift'] = test['tvalue'].shift(-1)

    # replace the last value NaN with the final date to produce 0 
    i = test['tvalue'][-1]
    test['shift'] = test['shift'].fillna(i)

    # delta is the difference between columns
    test['delta'] = test['tvalue'] - test['shift']

    # convert delta to minutes
    test['minutes'] = (test['delta'].dt.seconds.div(60).astype(int) 
                 + test['delta'].dt.days.multiply(1440).astype(int) )

    # take the average of delta excluding the last value (0)
    minutes_between_tweets = np.round(test.iloc[:-1,-1].mean(),decimals=0)

    minutes_between_tweets_results.append(minutes_between_tweets)

print(len(minutes_between_tweets_results))
average_minutes_between = pd.DataFrame({'screen_name':screen_names,
                                        'average_minutes_between_tweets':minutes_between_tweets_results})

4469


In [21]:
average_minutes_between.tail()

,screen_name,average_minutes_between_tweets
4464,IsIllinois,18.0
4465,Alberto42456937,441.0
4466,maudenificent,42.0
4467,doctorbenzene,71.0
4468,tracy_hixon,10.0


In [22]:
results = results.merge(average_minutes_between,left_on='screen_name',right_on='screen_name')
results.head()

,screen_name,non_unique_tweets,average_tweets_day,average_minutes_between_tweets
0,JohnSarich2,0,40.0,25.0
1,FranS199,0,33.0,37.0
2,mbnvcxz2,0,1.0,22610.0
3,Noles4LifeInSC,0,33.0,36.0
4,LifeCycleUKteam,0,3.0,743.0


In [23]:
# average hours tweeted each day
average_hours_tweeted = []
username = screen_names[0:10]

for i in screen_names:
    
    # slice by username
    test = pd.DataFrame(df.loc[df.index == i])
    test['time'] = pd.to_datetime(test.time, format='%H:%M:%S')
    test['hour'] = test['time'].dt.hour
    
    # pivot by date and hour of tweet
    table = pd.pivot_table(test, values=['tweetid'], index=['date'],
                        columns=['hour'], aggfunc='count',fill_value=0)
    
    # formatting
    table2 = table.reset_index()
    table2.columns = table2.columns.droplevel()
    table2.rename(columns={table2.columns[0]: "date" }, inplace = True)
    
    # replace the counts with 1 where tweets are greater than 0
    table3 = np.where(table2.iloc[:,1:] != 0 , 1, table2.iloc[:,1:])

    # return the average hours tweeted per day (normalised because all out of 24)
    hours = np.sum(table3,axis=1)
    mean_hours = np.round(np.mean(hours),decimals=0)

    average_hours_tweeted.append(mean_hours)
    
print(len(average_hours_tweeted))
average_hours_tweeted_df = pd.DataFrame({'screen_name':screen_names,
                                        'average_hours_tweeted':average_hours_tweeted})

4469


In [24]:
average_hours_tweeted_df.tail()

,screen_name,average_hours_tweeted
4464,IsIllinois,7.0
4465,Alberto42456937,3.0
4466,maudenificent,10.0
4467,doctorbenzene,6.0
4468,tracy_hixon,6.0


In [25]:
results = results.merge(average_hours_tweeted_df,left_on='screen_name',right_on='screen_name')
results.head()

,screen_name,non_unique_tweets,average_tweets_day,average_minutes_between_tweets,average_hours_tweeted
0,JohnSarich2,0,40.0,25.0,4.0
1,FranS199,0,33.0,37.0,10.0
2,mbnvcxz2,0,1.0,22610.0,1.0
3,Noles4LifeInSC,0,33.0,36.0,4.0
4,LifeCycleUKteam,0,3.0,743.0,2.0


In [26]:
# # number of tweets with same content per day not used atm

usernames = 'femmekatz'

for i in usernames: 
    test = df.loc[df.index == i]
    table = pd.pivot_table(test, values=['tweetid'], index=['text'],
                            columns=['date'], aggfunc='count',fill_value=0)

    table2 = pd.DataFrame(table)
    table2 = table2.reset_index()
    table2 = table2.iloc[:,1:]
    table3 = np.where(table2.iloc[:,0:] > 1)

In [27]:
results.to_csv("tweet_freqency_results.csv")